In [ ]:
import tqdm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

Here we'll look at a simplified model of interacting atoms that can exhibit solid, liquid, and gas-like behavior.
The *Lennard-Jones* potential is commonly used in molecular dynamics; it specifies that atoms repel each other at short distances, attract each other at moderate distances, and barely interact at all at long distances.
We'll write $r$ for the distance between two atoms.
The LJ potential is specified by two constants, an energy $\varepsilon$ and a characteristic distance $R$:
$$U = \varepsilon\left(\left(\frac{R}{r}\right)^{12} - 2\left(\frac{R}{r}\right)^6\right).$$
The force between two atoms is the derivative of the potential $U$.
At $r = R$, the force is zero, so this is the equilibrium distance.

In [ ]:
ε = 1.0
R = 1.0

In [ ]:
rs = np.linspace(0.0, 3 * R, 256)[1:]
Us = ε * ((R / rs)**12 - 2 * (R / rs)**6)

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("distance ($R$)")
ax.set_ylabel("energy ($\\varepsilon$)")
ax.set_ylim((-ε, +ε))
ax.plot(rs[1:], Us[1:]);

The functions below compute the energy and forces between a (possible large) collection of particles.

In [ ]:
def lennard_jones_potential(q):
    U = 0.0
    n = len(q)
    for i in range(n):
        for j in range(i + 1, n):
            z = q[i] - q[j]
            ρ = np.sqrt(np.dot(z, z)) / R
            U += ε / ρ ** 6 * (1 / ρ ** 6 - 2)

    return U


def lennard_jones_force(q):
    fs = np.zeros_like(q)
    n = len(q)
    for i in range(n):
        for j in range(i + 1, n):
            z = q[i] - q[j]
            ρ = np.sqrt(np.dot(z, z)) / R
            f = -12 * ε / R ** 2 / ρ ** 8 * (1 - 1 / ρ ** 6) * z
            fs[i] += f
            fs[j] -= f

    return fs

Here I've created an initial condition where the particles are laid out in a regular grid.

In [ ]:
num_rows, num_cols = 10, 10
num_particles = num_rows * num_cols

q = np.zeros((num_particles, 2))
for i in range(num_rows):
    for j in range(num_cols):
        q[num_cols * i + j] = (R * i, R * j)


σ = 0.1
rng = np.random.default_rng(seed=1729)
p = σ * rng.normal(size=(num_particles, 2))

In [ ]:
dt = 1e-2
final_time = 20.0
num_steps = int(final_time / dt)

This function implements a symplectic integration scheme, which approximately conserves energy.

In [ ]:
def semi_explicit_euler(q, p, dt, num_steps, force, progressbar=False):
    qs = np.zeros((num_steps + 1,) + q.shape)
    ps = np.zeros((num_steps + 1,) + p.shape)

    qs[0] = q
    ps[0] = p

    for t in tqdm.trange(num_steps):
        qs[t + 1] = qs[t] + dt * ps[t]
        ps[t + 1] = ps[t] + dt * force(qs[t + 1])
        
    return qs, ps

In [ ]:
qs, ps = semi_explicit_euler(q, p, dt, num_steps, lennard_jones_force)

The code below makes a movie.

In [ ]:
%%capture

fig, ax = plt.subplots()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.set_xlim((-2 * R, (num_rows + 1) * R))
ax.set_ylim((-2 * R, (num_cols + 1) * R))
ax.set_aspect("equal")
points = ax.scatter(qs[0, :, 0], qs[0, :, 1], animated=True)

def update(timestep):
    points.set_offsets(qs[timestep, :, :])

num_steps = len(qs)
fps = 60
animation = FuncAnimation(fig, update, num_steps, interval=1e3 / fps)

In [ ]:
HTML(animation.to_html5_video())

And a plot of the kinetic and potential energies.

In [ ]:
ts = np.linspace(0, final_time, num_steps)
Us = np.array([lennard_jones_potential(q) for q in qs]) / num_particles
Ks = 0.5 * np.sum(ps ** 2, axis=(1, 2)) / num_particles

In [ ]:
fig, ax0 = plt.subplots()
ax0.plot(ts, Us, color="tab:blue")
ax0.tick_params(axis="y", labelcolor="tab:blue")
ax0.set_ylabel("potential", color="tab:blue")
ax1 = ax0.twinx()
ax1.plot(ts, Ks, color="tab:orange")
ax1.tick_params(axis="y", labelcolor="tab:orange")
ax1.set_ylabel("kinetic", color="tab:orange");

In [ ]:
Es = Ks + Us
δE = Es.max() - Es.min()
δE / Ks.mean()